# Testing code

# Importing stuff

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import warnings

warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [2]:
import os
import os.path as osp
import random

import mlflow
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm_notebook

from nlpclass.config import model_config
from nlpclass.data.data_utils import TranslationDataset, text_collate_func
from nlpclass.models.evaluation_utils import bleu_eval, output_to_translations
from nlpclass.models.models import DecoderRNN, EncoderRNN, TranslationModel
from nlpclass.models.training_utils import load_data

In [3]:
CURRENT_PATH = os.getcwd()
DATA_DIR = osp.join(CURRENT_PATH, '..', 'data')
MODEL_DIR = osp.join(CURRENT_PATH, '..','models')

# Data

In [4]:
data, data_loaders, max_length = load_data('vi', batch_size=16)

Counting words...
Counted words:
eng 35151
vi 43279
Counting words...
Counted words:
eng 3562
vi 3678
Counting words...
Counted words:
eng 3361
vi 3518


In [18]:
encoder = EncoderRNN(
    data['train'].input_lang.n_words,
    embedding_size=300,
    hidden_size=128,
    num_layers=1,
    dropout=0.0,
    pretrained_embeddings=data['train'].input_lang.embeddings,
    bidirectional=True).to(model_config.device)
decoder = DecoderRNN(
    embedding_size=300,
    hidden_size=128,
    output_size=data['train'].output_lang.n_words,
    pretrained_embeddings=data['train'].output_lang.embeddings,
    attention=True).to(model_config.device)
translation_model = TranslationModel(encoder, decoder, teacher_forcing_ratio=1.0).to(model_config.device)

In [19]:
optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, translation_model.parameters()), 1e-3)

In [11]:
weight = torch.ones(translation_model.decoder.output_size).to(model_config.device)
weight[model_config.PAD_token] = 0
criterion = nn.CrossEntropyLoss(weight)

In [12]:
def calc_loss(logits, target, criterion):
    logits_flat = logits.view(-1, logits.size(-1))
    target_flat = target.view(-1, 1).squeeze()
    return criterion(logits_flat, target_flat)

In [28]:
for i in tqdm_notebook(range(25)):
    for batch in data_loaders['train']:
        optimizer.zero_grad()
        logits = translation_model(batch)
        loss = calc_loss(logits, batch['target'], criterion)
        loss.backward()
        translation_model.encoder.embedding.weight.grad[data['train'].input_lang.pretrained_inds] = 0
        translation_model.decoder.embedding.weight.grad[data['train'].output_lang.pretrained_inds] = 0
        clip_grad_norm_(filter(lambda p: p.requires_grad,
                                   translation_model.parameters()), model_config.grad_norm)
        optimizer.step() 
    original, translation = evaluate(translation_model, data, data_loaders, dataset_type='train')

tensor(4.6251, device='cuda:0')
0.5936727299251386
tensor(3.3127, device='cuda:0')
1.6027140964129813
tensor(2.8484, device='cuda:0')
2.292629685459052
tensor(2.2910, device='cuda:0')
4.914684249301489
tensor(1.9952, device='cuda:0')
5.262287046642645
tensor(1.5924, device='cuda:0')
7.016682515114114
tensor(1.5501, device='cuda:0')
8.582478523753394
tensor(1.2819, device='cuda:0')
8.681783192243731
tensor(1.3297, device='cuda:0')
9.556247002276868
tensor(1.3010, device='cuda:0')
9.568101622144765
tensor(1.3031, device='cuda:0')
10.58796197851855
tensor(0.9066, device='cuda:0')
11.174392798479518
tensor(1.0672, device='cuda:0')
12.892230064192653
tensor(1.1184, device='cuda:0')
12.717057652800923
tensor(0.8126, device='cuda:0')
13.374455763125784
tensor(1.1716, device='cuda:0')
12.64711309856782
tensor(0.8032, device='cuda:0')
12.901827172145309
tensor(0.8611, device='cuda:0')
13.684486609129518
tensor(0.9717, device='cuda:0')
12.75010872298263
tensor(0.8819, device='cuda:0')
11.6897909

In [11]:
x = next(iter(data_loaders['train']))
input_seq = x['input']
target_seq = x['target']
input_length = x['input_length']
target_length = x['target_length']

encoded_input, hidden = translation_model.encoder.forward(input_seq, input_length)

In [13]:
x = next(iter(data_loaders['train']))

optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, translation_model.parameters()), 1e-2)

for i in range(100):
    optimizer.zero_grad()
    logits = translation_model(x)
    loss = calc_loss(logits, x['target'], criterion)
    print(loss)
    loss.backward()
    translation_model.encoder.embedding.weight.grad[data['train'].input_lang.pretrained_inds] = 0
    translation_model.decoder.embedding.weight.grad[data['train'].output_lang.pretrained_inds] = 0
    clip_grad_norm_(filter(lambda p: p.requires_grad,
                               translation_model.parameters()), model_config.grad_norm)
    optimizer.step()

tensor(11.2637, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(10.2678, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(7.5081, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.9124, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.1691, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(4.0291, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.7860, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.5029, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.2462, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(3.0262, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.8312, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.6644, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.5155, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.3773, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2680, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.1681, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.0610, device='cuda:0', grad_fn=<NllLossBackwa

In [14]:
original = output_to_translations(x['target'], data['train'])
translations = output_to_translations(translation_model.greedy(x), data['train'])
bleu_eval(original, translations)

100.00000000000004

In [15]:
def evaluate(model, data, data_loaders, dataset_type='dev', max_batches=100):
    model.eval()
    with torch.no_grad():
        original_strings = []
        translated_strings = []
        for i, batch in enumerate(data_loaders[dataset_type]):
            if i > max_batches:
                break
            logits = translation_model(batch)
            epoch_loss = calc_loss(logits, batch['target'], criterion)
            original = output_to_translations(batch['target'], data['train'])
            translations = output_to_translations(model.greedy(batch), data['train'])
            original_strings.extend(original)
            translated_strings.extend(translations)
        bleu = bleu_eval(original_strings, translated_strings)
        model.train()
        print(epoch_loss)
        print(bleu)
        
        return original_strings, translated_strings

In [20]:
optimizer = torch.optim.Adam(
            filter(lambda p: p.requires_grad, translation_model.parameters()), 1e-3)
weight = torch.ones(translation_model.decoder.output_size).to(model_config.device)
weight[model_config.PAD_token] = 0
criterion = nn.CrossEntropyLoss(weight)

In [ ]:
for i, batch in enumerate(tqdm_notebook(data_loaders['train'])):
    if i % 500 == 0:
        evaluate(translation_model, data, data_loaders)
        #evaluate(translation_model, data, data_loaders, dataset_type='train')
    optimizer.zero_grad()
    logits = translation_model(batch)
    loss = calc_loss(logits, batch['target'], criterion)
    loss.backward()
    translation_model.encoder.embedding.weight.grad[data['train'].input_lang.pretrained_inds] = 0
    translation_model.decoder.embedding.weight.grad[data['train'].output_lang.pretrained_inds] = 0
    clip_grad_norm_(filter(lambda p: p.requires_grad,
                               translation_model.parameters()), model_config.grad_norm)
    optimizer.step()

tensor(10.6677, device='cuda:0')
3.58132009002981e-05


In [ ]:
original_strings, translated_strings = evaluate(translation_model, data, data_loaders)

In [13]:
original_strings[0:10]

['baseball be later but of volumes all. hope. answer. only were',
 'be this. 65. know what must truth. i most',
 'games be practical everybody pins your needles all forward careful were',
 'i moment. it attack. be sorrows. ten.',
 'canadian be appointment careful laugh. no truth. something time were',
 'beautiful cupboard. hot. wine these',
 'much. but how bitter really dishonesty fed recently',
 'applied all else fed to climate husbands. matter. key',
 'but they answer. perfect. today. tape',
 'i is invented i situation getting powers your few']

In [14]:
translated_strings[0:10]

['i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the',
 'i you you to the']

In [48]:
decoder_input.size()

torch.Size([16])

In [23]:
total_loss.backward()

In [30]:
predictions = translation_model.greedy(x)

In [215]:
for row in predictions.cpu().numpy():
    decoded_words = []
    for elem in row[1:]:
        decoded_words.append(data['train']['output_lang'].index2word[elem])
        if elem == model_config.EOS_token:
            break

In [167]:
yo = Variable(torch.LongTensor([model_config.SOS_token] * 8)).to(model_config.device)
yo = torch.stack((yo, topi.squeeze(), topi.squeeze()), dim=1)

In [99]:
total_loss.backward()

In [57]:
seq_range = torch.autograd.Variable(torch.LongTensor(np.repeat([2], len(x['input_length'])))).to(model_config.device)
mask = seq_range < x['input_length']
loss = -torch.gather(decoder_output, dim=1, index=input_var.unsqueeze(1)).squeeze() * mask.float()

In [66]:
loss.sum() / torch.sum(loss > 0).float()

tensor(11.4193, device='cuda:0', grad_fn=<DivBackward1>)

In [63]:
torch.sum(loss > 0).cpu().numpy()

array(8)

In [ ]:
encoder_output, encoder_hidden = encoder(x['input'], x['input_length'])

In [290]:
context = None
if decoder.attention:
    context = Variable(torch.zeros(encoder_output.size(0), encoder_output.size(2))).unsqueeze(1).to(model_config.device)

In [291]:
decoder_output, decoder_hidden, context, weights = decoder(input_var, encoder_hidden, encoder_output, context)

In [16]:
def train_model(model, optimizer, train_loader, criterion):
    model.train()
    loss_train = 0
    for batch in train_loader:
        optimizer.zero_grad()
        outputs = model(batch)
        loss = criterion(outputs, batch['label'])
        loss.backward()
        optimizer.step()
        loss_train += loss.item() * \
            len(batch['label']) / len(train_loader.dataset)
    return loss_train

In [17]:
optimizer = torch.optim.Adam(translation_model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss()
train_model(translation_model, optimizer, train_loader, criterion)

KeyError: 'label'